# Uber Fares

Dataset je úspešne ukradnutý z [Kaggle](https://www.kaggle.com/datasets/yasserh/uber-fares-dataset/data). Sťahovať ho môžete [odtiaľto](data/uber.csv).

![obrazok](images/uber.jpg)

## Načítanie datasetu

In [1]:
import pandas as pd

df = pd.read_csv('data/uber.csv')

okrem načítania:
* premenujeme niekoľko stĺpcov
* pretypujeme stĺpce na menšie údajové typy
* odstránime duplicitný stĺpec
* odstránime riadky, ktoré nemajú hodnotu

In [2]:
# cleanup a priprava

## Prehľad

In [3]:
# pohlad na vlastnosti datasetu
df

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5
...,...,...,...,...,...,...,...,...,...
199995,42598914,2012-10-28 10:49:00.00000053,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,1
199996,16382965,2014-03-14 01:09:00.0000008,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1
199997,27804658,2009-06-29 00:42:00.00000078,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,2
199998,20259894,2015-05-20 14:56:25.0000004,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,1


## Otázky

1. Koľko peňazí sme zarobili za dané obdobie?

In [4]:
# tak kolko?

2. Koľko peňazí sme zarobili za jednotlivé roky?
   Poznámka: tu môžete nakresliť aj pekný obrázok

In [5]:
# tak kolko?

3. V ktorom mesiaci v roku 2013 bolo najviac jázd?

In [6]:
# tak v ktorom?

4. V ktorom mesiaci v roku 2014 bolo najviac jázd?

In [7]:
# tak v ktorom?

5. V ktorý deň za uvedené obdobie bolo prevezných najviac pasažierov?

In [8]:
# tak v ktorý?